## Initial Lib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## Keypoint Mediapipe

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

## Dataset

In [3]:
DATA_PATH = (r"C:\Users\victus\Documents\SignLanguage\MotionDetectionforSIBIv2\DatasetV2")
doc = []

for filename in os.listdir(DATA_PATH):
    label = filename.split(".")[0]
    doc.append(label)

actions = np.array(doc)

no_sequences = 30

sequence_length = 30

start_folder = 30

In [4]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'aku': 0,
 'apa': 1,
 'bagaimana': 2,
 'berapa': 3,
 'di': 4,
 'F': 5,
 'halo': 6,
 'I': 7,
 'J': 8,
 'K': 9,
 'kamu': 10,
 'kapan': 11,
 'ke': 12,
 'mana': 13,
 'mereka': 14,
 'nama': 15,
 'saya': 16,
 'siapa': 17,
 'Y': 18,
 'yang': 19}

## Train Test

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}_keypoints_{}.npy'.format(action, frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
np.array(sequences).shape

(600, 30, 126)

In [7]:
np.array(labels).shape

(600,)

In [8]:
X = np.array(sequences)

In [9]:
X.shape

(600, 30, 126)

In [10]:
y = to_categorical(labels).astype(int)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [43]:
y_test.shape

(90, 20)

## Build Model

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import  Input, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback

In [45]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [46]:
class StopTrainingCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch > 200 and logs.get('loss') < 0.09 and (logs.get('categorical_accuracy') < 0.99 and logs.get('categorical_accuracy') > 0.91):
            print("Stopping training")
            self.model.stop_training = True
            
stop_callback = StopTrainingCallback()

In [47]:
lstm_input = Input(shape=(30, 126))

In [59]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(512, return_sequences=True, activation='relu'))
model.add(LSTM(256, return_sequences=False, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [60]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback, stop_callback]) #

Epoch 1/500
16/16 [==============================] - 6s 239ms/step - loss: 2.8215 - categorical_accuracy: 0.1137
Epoch 2/500
16/16 [==============================] - 4s 234ms/step - loss: 3.4138 - categorical_accuracy: 0.2294
Epoch 3/500
16/16 [==============================] - 4s 254ms/step - loss: 2.6889 - categorical_accuracy: 0.1882
Epoch 4/500
16/16 [==============================] - 4s 259ms/step - loss: 2.4311 - categorical_accuracy: 0.1588
Epoch 5/500
16/16 [==============================] - 4s 264ms/step - loss: 2.3306 - categorical_accuracy: 0.1157
Epoch 6/500
16/16 [==============================] - 4s 255ms/step - loss: 2.2054 - categorical_accuracy: 0.1745
Epoch 7/500
16/16 [==============================] - 4s 246ms/step - loss: 2.0856 - categorical_accuracy: 0.2725
Epoch 8/500
16/16 [==============================] - 4s 244ms/step - loss: 1.9661 - categorical_accuracy: 0.3216
Epoch 9/500
16/16 [==============================] - 4s 249ms/step - loss: 31.9773 - categorical

16/16 [==============================] - 4s 243ms/step - loss: 0.2233 - categorical_accuracy: 0.9353
Epoch 73/500
16/16 [==============================] - 4s 254ms/step - loss: 0.1418 - categorical_accuracy: 0.9549
Epoch 74/500
16/16 [==============================] - 4s 258ms/step - loss: 0.1975 - categorical_accuracy: 0.9294
Epoch 75/500
16/16 [==============================] - 4s 254ms/step - loss: 0.2144 - categorical_accuracy: 0.9412
Epoch 76/500
16/16 [==============================] - 4s 249ms/step - loss: 0.3889 - categorical_accuracy: 0.8961
Epoch 77/500
16/16 [==============================] - 4s 254ms/step - loss: 0.2652 - categorical_accuracy: 0.9000
Epoch 78/500
16/16 [==============================] - 4s 277ms/step - loss: 0.1446 - categorical_accuracy: 0.9529
Epoch 79/500
16/16 [==============================] - 4s 268ms/step - loss: 0.1225 - categorical_accuracy: 0.9569
Epoch 80/500
16/16 [==============================] - 4s 278ms/step - loss: 0.1157 - categorical_accu

16/16 [==============================] - 4s 261ms/step - loss: 0.2004 - categorical_accuracy: 0.9569
Epoch 144/500
16/16 [==============================] - 4s 256ms/step - loss: 0.1176 - categorical_accuracy: 0.9725
Epoch 145/500
16/16 [==============================] - 4s 249ms/step - loss: 0.0713 - categorical_accuracy: 0.9843
Epoch 146/500
16/16 [==============================] - 4s 247ms/step - loss: 0.0897 - categorical_accuracy: 0.9725
Epoch 147/500
16/16 [==============================] - 4s 245ms/step - loss: 0.1327 - categorical_accuracy: 0.9608
Epoch 148/500
16/16 [==============================] - 4s 248ms/step - loss: 0.1225 - categorical_accuracy: 0.9745
Epoch 149/500
16/16 [==============================] - 4s 246ms/step - loss: 0.4071 - categorical_accuracy: 0.8510
Epoch 150/500
16/16 [==============================] - 4s 249ms/step - loss: 0.2605 - categorical_accuracy: 0.9118
Epoch 151/500
16/16 [==============================] - 4s 247ms/step - loss: 0.1604 - categori

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 256)           392192    
                                                                 
 lstm_10 (LSTM)              (None, 30, 512)           1574912   
                                                                 
 lstm_11 (LSTM)              (None, 256)               787456    
                                                                 
 dense_9 (Dense)             (None, 256)               65792     
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 20)                2580      
                                                                 
Total params: 2,855,828
Trainable params: 2,855,828
No

## Matrix and Accuracy

In [52]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [53]:
yhat = model.predict(X_test)

In [54]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [55]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[86,  0],
        [ 0,  4]],

       [[85,  0],
        [ 0,  5]],

       [[86,  0],
        [ 0,  4]],

       [[86,  0],
        [ 0,  4]],

       [[86,  0],
        [ 1,  3]],

       [[85,  0],
        [ 0,  5]],

       [[84,  0],
        [ 0,  6]],

       [[87,  1],
        [ 0,  2]],

       [[83,  0],
        [ 1,  6]],

       [[87,  0],
        [ 0,  3]],

       [[86,  0],
        [ 0,  4]],

       [[87,  0],
        [ 0,  3]],

       [[86,  0],
        [ 0,  4]],

       [[85,  0],
        [ 0,  5]],

       [[85,  0],
        [ 0,  5]],

       [[83,  1],
        [ 0,  6]],

       [[85,  1],
        [ 0,  4]],

       [[87,  0],
        [ 0,  3]],

       [[84,  0],
        [ 2,  4]],

       [[83,  1],
        [ 0,  6]]], dtype=int64)

In [56]:
accuracy_score(ytrue, yhat)

0.9555555555555556

## Save and Reload Model

In [57]:
model.save('realtimeV10.h5')

In [58]:
del model

In [61]:
model.load_weights('realtimeV10.h5')

## Real Time test

In [62]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        height, width, channels = image.shape
        text_size, _ = cv2.getTextSize(' '.join(sentence), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_width = text_size[0]
        x = (width - text_width) // 2
        cv2.putText(image, ' '.join(sentence), (x, height - 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_bas

<class 'mediapipe.python.solution_base.SolutionOutputs'>
saya
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base.SolutionOutputs'>
di
<class 'mediapipe.python.solution_base